# Step 1: Authentication

```markdown
# Overview
This is the first of three challenges to learn about Azure AI Agent Service. 
You will create a simple conversational AI agent using Azure AI Projects. 
This agent will engage in a basic conversation without any tools or external integrations.

## You will:
- Set up an Azure AI Agent.
- Allow the user to chat with the agent interactively.
- Experiment with modifying the agent's behavior.

## Task
Try modifying the `instructions` given to the agent and experiment with different parameters, such as the model used (`gpt-4o-mini`) or the way responses are structured. 
See how the agent’s responses change based on different instructions!
```

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
from pathlib import Path

# ---------------------------------------------------------------------------
# Load Environment Variables
# ---------------------------------------------------------------------------

# Load environment variables from the .env file
env_path = Path().resolve().parent.parent / ".env"
load_dotenv(dotenv_path=env_path)

# Retrieve required environment variables
PROJECT_CONNECTION_STRING = os.getenv("Azure_AI_PROJECT_CONNECTION_STRING")

# Ensure that required variables are set
if not PROJECT_CONNECTION_STRING:
    raise ValueError("PROJECT_CONNECTION_STRING is not set in the .env file.")
print("Environment variables loaded successfully.")

Environment variables loaded successfully.


# Step 2: Defining and Testing the Agent

```markdown
Next we define the agent. We won't add any tools yet. This is merely for validating that we have set up the correct keys and can call the service.
Each agent starts in a thread. Threads keep track of the conversation history and the tools available to an agent. We can manage threads via their thread id.

```

In [2]:
# ---------------------------------------------------------------------------
# Prepare the tool for the Azure AI Agent service.
# ---------------------------------------------------------------------------


"""
Initializes and runs a simple Azure AI Agent for interactive conversation.
"""
# Create the Azure AI Projects client using Azure CLI authentication
client = AIProjectClient.from_connection_string(
    credential=AzureCliCredential(),
    conn_str=PROJECT_CONNECTION_STRING,
)

# Instructions for the agent
instructions = """
You are a helpful AI assistant. Engage in a conversation and provide informative responses.
"""

with client:
    # Create the agent using a predefined model (e.g., gpt-4o-mini)
    # Option: Experiment with a different model such as gpt-4o (make sure the model is available under this name).
    agent = client.agents.create_agent(
        model="gpt-4o-mini",  # Change to "gpt-4" or another supported model to see differences.
        name="chat-agent",
        instructions=instructions
    )

    # Start a new conversation thread
    # This thread holds the conversation history. By default, all messages accumulate here.
    thread = client.agents.create_thread()
    # store thread id to csv
    thread_id = thread.id
    print(f"Created thread, thread ID: {thread_id}")
    print("Type 'exit' or 'quit' to end the conversation.")
    print("Please enter your message:")
    import time

    last_message_id = None  # Store the last processed message ID

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        # Send user message
        client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_input
        )

        # Process the conversation
        run = client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )

        # Wait for the run to complete
        while run.status not in ["completed", "failed"]:
            time.sleep(1)
            run = client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "failed":
            print("Assistant: Sorry, I encountered an error.")
            continue

        # Retrieve latest messages
        messages = client.agents.list_messages(thread_id=thread.id)["data"]

        # Find the latest assistant message (skip already printed ones)
        for msg in sorted(messages, key=lambda x: x["created_at"], reverse=True):
            if msg["role"] == "assistant" and msg["id"] != last_message_id:
                last_message_id = msg["id"]  # Update last processed message ID
                text_content = next(
                    (part["text"]["value"] for part in msg["content"] if part["type"] == "text"), None
                )
                if text_content:
                    print("Assistant:", text_content)
                break  # Stop after printing the latest message

                
    # Cleanup: Delete the agent after execution
    client.agents.delete_agent(agent.id)



Created thread, thread ID: thread_jO0EEuH0efh4ufIDg0J0tz5t
Type 'exit' or 'quit' to end the conversation.
Please enter your message:
Assistant: Hello! How can I assist you today?
Goodbye!


# Step 3: Retrieve the latest assistant message

```markdown
To resume a conversation you can simply use the ID of the thread
This is also useful for debugging and logging.
```

In [3]:
client = AIProjectClient.from_connection_string(
        credential=AzureCliCredential(),
        conn_str=PROJECT_CONNECTION_STRING,
    )
    
print("retrieving messages for thread id: ", thread_id)
with client:
        # Create the agent using a predefined model (e.g., gpt-4o-mini)
        # Option: Experiment with a different model such as gpt-4o (make sure the model is available under this name).

    messages = client.agents.list_messages(thread_id=thread_id)
    if len(messages.data) == 0:
        print("No messages found.")
        exit(0)
    # The messages are following in the reverse order,
    # we will iterate them and output only text contents.
    for data_point in reversed(messages.data):
        last_message_content = data_point.content[-1]
        print(data_point.role, last_message_content)


retrieving messages for thread id:  thread_jO0EEuH0efh4ufIDg0J0tz5t
MessageRole.USER {'type': 'text', 'text': {'value': 'hi', 'annotations': []}}
MessageRole.AGENT {'type': 'text', 'text': {'value': 'Hello! How can I assist you today?', 'annotations': []}}


In [4]:
# this resumes an existing thread
client = AIProjectClient.from_connection_string(
        credential=AzureCliCredential(),
        conn_str=PROJECT_CONNECTION_STRING,
    )
thread = client.agents.get_thread(thread_id)